In [ ]:
from clustering import *
from utils import *

In [ ]:
#df of log returns
df_60m = pd.read_parquet("data/clean/60m/data_clean_60m_log_ret.parquet")

In [ ]:
#Rolling clustering for 5m dataset
#Cluster days
#Compute T=N/3
N = df_60m.shape[1]
T = N // 3

#Number of windows possible
lag_max = df_60m.shape[0] - T
liste = RolledCluster(df_60m, T, "louvain_cluster/date/60m", True, False, lag_max)

In [ ]:
liste_cluster = dask.compute(dask.compute(liste))[0][0]
#liste_cluster

In [ ]:
number_of_clusters = []
for element in liste_cluster:
    number_of_clusters.append(len(element["Cluster"].unique()))

In [ ]:
df_n_clusters = pd.DataFrame(number_of_clusters, columns=['Clusters'])
df_n_clusters["Datetime"] = df_60m[-len(df_n_clusters):].index

In [ ]:
df_n_clusters

In [ ]:
plot(df_n_clusters, x="Datetime", y="Clusters", title='Number of clusters per period', save_name='Plot_number_clusters_days_60m')

In [ ]:
df_n_clusters["ARI"] = [1] + RolledARI(liste_cluster)

In [ ]:
plot(df_n_clusters, x="Datetime", y="ARI", title='ARI measure with respect to time', save_name='Plot_ARI_days_60m')

In [ ]:
liste_cluster[-1].value_counts()

In [ ]:
dfj = df_60m.join(liste_cluster[-1], how='inner')

In [ ]:
dfj

In [ ]:
dfj = dfj.groupby(by='Cluster').mean().T

In [ ]:
asset_details = pd.read_csv('tickers/s&p.csv', index_col=0)
asset_details

In [ ]:
dfj = dfj.join(asset_details[['Sector']], how='inner')
dfj

In [ ]:
plot_scatter(df=dfj, x=0, y=1, hue='Sector', title='Average log returns per asset per cluster (60m data)')